In [ ]:
# default_exp portalcatalog

# portalcatalog
> API Details


In [ ]:
#export
import urllib
import gzip
import json
import pandas as pd
from functools import partial

In [ ]:
#export

def read_gz(url:str):
    response = urllib.request.urlopen(url)
    decompressed_bytes = gzip.decompress(response.read())
    text = decompressed_bytes.decode('utf-8')
    return text

def get_jsonlines(jsonl_content: str):
    return [json.loads(jline) for jline in jsonl_content.split('\n') if jline]

In [ ]:
#export
def get_value_for_language(lang: str, value_dict: dict):
    return value_dict.get(lang, None)
    
def get_datasets(jsonl_content: str):
    return [CatalogDataset(json.loads(jline)) for jline in jsonl_content.split('\n') if jline]
    
class CatalogDataset:
    
    """
    A dataset
    """
    def __init__(self, record=None, language:str = 'en'):
        self.record = record
        self.resources = [CatalogResource(record, language) for record in record['resources']]
        self.language = language
        for key, value in record.items():
            if value:
                if isinstance(value, dict):
                    value_fn = partial(get_value_for_language, value_dict=value)
                    setattr(self, f'get_{key}', value_fn)
    
    def get_org(self):
        if hasattr(self, 'get_org_title_at_publication'):
            return self.get_org_title_at_publication(self.language)
        return 'Organization'
    
    def get_name(self):
        if hasattr(self, 'get_data_series_name'):
            return self.get_data_series_name(self.language)
        return f'{self.get_org()} Dataset'
                    
    def get_notes(self, language:str = None):
        language = language or self.language
        return self.get_notes_translated(lang=language)
    
    def resources_as_html(self):
        df= pd.DataFrame([{'name': resource.get_name(), 'state': resource.get_state()} 
                          for resource in self.resources])
        return df.to_html(index=False)
        
    
    def __repr__(self):
        return f'Dataset {self.get_name()}'
    
    def _repr_html_(self):
        html = f'<h3>{self.get_name()}</h3>'
        html += f'<span>{self.get_notes()}</span>'
        html += pd.DataFrame([{'Publisher': self.get_org()}]).to_html(index=False, justify='left')
        html += '<h3>Resources</h3>'
        html += self.resources_as_html()
        return html
    
class CatalogResource:
    
    """
    A resource. Belongs to a dataset, which will have 1 or more resources
    """
    def __init__(self, record, language:str = 'en'):
        self.record = record
        self.id = record.get('id')
        self.language = language
    
    def get_name(self):
        return get_value_for_language(self.language, self.record.get('name_translated'))
    
    def get_state(self):
        return self.record.get('state', '')
    
    def get_url(self):
        return self.record.get('url', '')
    
    def __repr__(self):
        return f'{self.get_name()}'
    
    def _repr_html_(self):
        html = f'<h3>{self.get_name()}</h3>'
        return html
    
    
class PortalCatalog:
    
    """
    A portal catalog
    """
    def __init__(self, jsonl_content: str):
        self.datasets = get_datasets(jsonl_content)
    
    @classmethod
    def from_jsonl(cls, jsonl_content: str):
        return cls(jsonl_content)

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_config.ipynb.
Converted 01_core.ipynb.
Converted 02_text.ipynb.
Converted 03_inventory.ipynb.
Converted 04_repo.ipynb.
Converted 05_datatools.ipynb.
Converted 06_datasets.ipynb.
Converted 07_statscan.ipynb.
Converted 08_portalcatalog.ipynb.
Converted index.ipynb.
